# Quickstart

This is a guide to using the ATMO equilibrium chemistry plugin for TauREx 3.1, developed by Michelle Fabienne Bieger and Ahmed Al-Refaie.

## Prerequisites

Before using this plugin, please ensure that you have already installed TauREx (https://taurex3-public.readthedocs.io/; please note you will also need absorption cross sections) and ATMO (https://www.gitlab.erc-atmo.eu/1-2d_tools/atmo). To receive access to ATMO, please contact Pascal Tremblin of CEA Saclay: pascal.tremblin@cea.fr 

In [1]:
# Check that you have the latest version of TauREx (3.1.1-alpha)
# Git-clone the UCL GitHub page in order to get the latest version 

import taurex
print(taurex.__version__)

3.1.1-alpha


Now let's see if your frogr has installed correctly: 

In [3]:
# Check that you have installed frogr correctly
# Note that the plugin itself is named atmopy 
import atmopy
# print(atmopy.__version__)

Make sure to set the path to an established ATMO .exe! 

In [ ]:
import os
os.environ["ATMO_PATH"]="/Users/mbieger/atmo/tutorial/chem_eq/"

In [ ]:
# Importing required interactive plotting packages 

import matplotlib.pyplot as plt
%matplotlib notebook
from ipywidgets import *
import numpy as np
import pickle
import sys

In [ ]:
# Load required TauREx cross sections
# Customise to your system paths as required 
from taurex.cache import OpacityCache,CIACache

OpacityCache().clear_cache()
OpacityCache().set_opacity_path("/Users/mbieger/data/xsec/xsec_sampled_R15000_0")
CIACache().set_cia_path("/Users/mbieger/data/cia/HITRAN/data")

# Showcasing the use of the ATMOPy plugin 

In this section, we'll be exploring how to use the ATMOPy plugin. First, we'll set up a TauREx model and then insert the ATMOPy chemistry to see how that affects the forward model and subsequent retrievals.

### Setting up the forward model 

In [ ]:
# Select a temperature profile
from taurex.temperature import Isothermal
iso = Isothermal(T=1900.0)

# Setting up the planet
from taurex.planet import Planet
planet = Planet(planet_radius=1.53,planet_mass=0.85)

# Setting up a host blackbody star
from taurex.stellar import BlackbodyStar
star = BlackbodyStar(temperature=6600.0,radius=1.51)

# Setting up a chemistry profile: we want to use our newfound fun toy
from atmopy.taurex import ATMOChemistry

# Initialising a transmission model with pressures and layers and incorporating our previously loaded profiles
from taurex.model import TransmissionModel
tm = TransmissionModel(planet=planet,
                       temperature_profile=iso,
                       chemistry=ATMOChemistry(),
                       star=star,
                       atm_min_pressure=1e-0,
                       atm_max_pressure=1e6,
                       nlayers=30)

In [ ]:
# Adding in actual physics to the model now with absorptions 
from taurex.contributions import AbsorptionContribution
tm.add_contribution(AbsorptionContribution())

# CIA absorption
from taurex.contributions import CIAContribution
tm.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))

# Rayleigh scattering
from taurex.contributions import RayleighContribution
tm.add_contribution(RayleighContribution())

Now we can build the transmission build and run it. Res ("result") of the transmission model will have four components (grouped via arrays) to it: 

- The wavenumber grid
- The *native* flux
- The optical depth
- Any extra information

In [ ]:
# Building the transmission model up
tm.build()

# Running the transmission model
res = tm.model()
res

In [ ]:
list(tm.fittingParameters.keys())

### Plot the chemistry of the atmosphere

In [ ]:
plt.figure()

for x,gasname in enumerate(tm.chemistry.activeGases):
    plt.plot(tm.chemistry.activeGasMixProfile[x],tm.pressureProfile,label=gasname)

# for x,gasname in enumerate(tm.chemistry.inactiveGases):
#     plt.plot(tm.chemistry.inactiveGasMixProfile[x],tm.pressureProfile/1e5,label=gasname)

plt.xscale("log")
plt.xlabel("Gas abundance [mole fraction]")

# plt.gca().invert_yaxis()
plt.yscale("log")
plt.ylabel("Pressure profile [Pa]")

plt.title("Active Gas Profiles")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,15))

for x,gasname in enumerate(tm.chemistry.inactiveGases):
    plt.plot(tm.chemistry.inactiveGasMixProfile[x],tm.pressureProfile/1e5,label=gasname)

plt.xscale("log")
plt.xlabel("Gas abundance [mole fraction]")

# plt.gca().invert_yaxis()
plt.yscale("log")
plt.ylabel("Pressure profile [Pa]")

plt.title("Inactive Gas Profiles")
plt.legend()
plt.show()

### Plot the flux of the atmosphere (and bin it!)

In [ ]:
native_grid, rprs, tau, _ = res

full_fig = plt.figure()
plt.plot(np.log10(10000/native_grid),rprs)

plt.xlabel(r"Wavelength ($\mu$m)")
plt.ylabel(r"(R$_p$/R$_s)^2$")
plt.title("Spectra")
plt.show()

In [ ]:
from taurex.binning import FluxBinner,SimpleBinner
binned_fig = plt.figure()


# Make a logarithmic grid
wngrid = np.sort(10000/np.logspace(-0.4,1.1,1000))
bn = SimpleBinner(wngrid=wngrid)

bin_wn, bin_rprs,_,_  = bn.bin_model(tm.model(wngrid=wngrid))

plt.plot(10000/bin_wn,bin_rprs)

plt.xscale('log')
plt.xlabel(r"Wavelength ($\mu$m)")
plt.ylabel(r"(R$_p$/R$_s)^2$")
plt.title("Binned spectra")

plt.show()

### Transmission, emission, and direct imaging too

In [ ]:
# Setting up emission and direct image, in the same vein as the TM above 
from taurex.model import EmissionModel, DirectImageModel
em = EmissionModel(planet=planet,
                       temperature_profile=iso,
                       chemistry=ATMOChemistry(),
                       star=star,
                        atm_min_pressure=1e-0,
                       atm_max_pressure=1e6,
                       nlayers=30)
di = DirectImageModel(planet=planet,
                       temperature_profile=iso,
                       chemistry=ATMOChemistry(),
                       star=star,
                        atm_min_pressure=1e-0,
                       atm_max_pressure=1e6,
                       nlayers=30)

em.add_contribution(AbsorptionContribution())
em.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))
em.add_contribution(RayleighContribution())

di.add_contribution(AbsorptionContribution())
di.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))
di.add_contribution(RayleighContribution())

em.build()
di.build()

In [ ]:
wngrid = np.sort(10000/np.logspace(-0.4,1.1,1000))

all_fig = plt.figure(figsize=(12,8))
tm_ax = all_fig.add_subplot(1,3,1)
em_ax = all_fig.add_subplot(1,3,2)
di_ax = all_fig.add_subplot(1,3,3)
model_tm, = tm_ax.plot(10000/wngrid,bn.bin_model(tm.model(wngrid))[1])
model_em, = em_ax.plot(10000/wngrid,bn.bin_model(em.model(wngrid))[1])
model_di, = di_ax.plot(10000/wngrid,bn.bin_model(di.model(wngrid))[1])
tm_ax.set_xscale('log')
em_ax.set_xscale('log')
di_ax.set_xscale('log')
tm_ax.set_title('Transmission')
em_ax.set_title('Emission')
di_ax.set_title('Direct Image')
tm_ax.set_xlabel(r"Wavelength ($\mu$m)")
em_ax.set_xlabel(r"Wavelength ($\mu$m)")
di_ax.set_xlabel(r"Wavelength ($\mu$m)")


# TauREx can iteratively change the model as you go with update_model

### Retrievals

In [ ]:
# Retrievals required data! Let's load her in 
from taurex.data.spectrum.observed import ObservedSpectrum
obs = ObservedSpectrum('/Users/mbieger/OneDrive - University of Exeter/planetspectra/wasp_79b/spectra/WASP-79_transmission_hstspitz.dat')

In [ ]:
# Binning down the native data 
obin = obs.create_binner()

# Plotting the data 
plt.figure(figsize=(8,6))
plt.errorbar(obs.wavelengthGrid,obs.spectrum,obs.errorBar,lw=1,color='black',marker='d',ls='none',zorder=1,label='Observed')

plt.plot(obs.wavelengthGrid,obin.bin_model(tm.model(obs.wavenumberGrid))[1],label='Transmission forward model')

plt.xlabel(r"Wavelength ($\mu$m)")
plt.ylabel(r"(R$_p$/R$_s)^2$")
plt.legend(loc='best')
plt.show()

In [ ]:
# Choosing the parameter estimation sampler: Nestle 

from taurex.optimizer.nestle import NestleOptimizer
opt = NestleOptimizer(num_live_points=50)

# Giving it our forward model (transmission) and the observations 
opt.set_model(tm)
opt.set_observed(obs)

# Giving it our preferred parameters to fit and their prior boundaries 
opt.enable_fit('planet_radius')
opt.enable_fit('T')
opt.set_boundary('T',[1200,2300])
opt.set_boundary('planet_radius',[0.5,2])

In [ ]:
# Fitting the retrieval 
solution = opt.fit()
taurex.log.disableLogging()

In [ ]:
# Plotting the solution via loop 
for solution,optimized_map,optimized_value,values in opt.get_solution():
    opt.update_model(optimized_map)
    plt.figure()
    plt.errorbar(obs.wavelengthGrid,obs.spectrum,obs.errorBar,lw=1,color='black',marker='d',ls='none',zorder=1,label='Observed')
    plt.plot(obs.wavelengthGrid,obin.bin_model(tm.model(obs.wavenumberGrid))[1],label='Retrieved transmission model')
    plt.xlabel(r"Wavelength ($\mu$m)")
    plt.ylabel(r"(R$_p$/R$_s)^2$")
    plt.legend()
    plt.show()

# Compare/contrasting to other TauREx chemistry models

Remember the models we've just set up? Let's see how it ATMO chemistry compares to other available chemistry plugin models.

In [ ]:
# from taurex.chemistry import TaurexChemistry
# chemistry = TaurexChemistry(fill_gases=['H2','He'],ratio=0.172)